In [1]:
import sys, os

# Enable importing pysparky
sys.path.append(os.pardir)

In [3]:
import pyspark
from pyspark.sql import SparkSession, DataFrame, Column
from pyspark.sql import functions as F, types as T

print(pyspark.__version__)

spark = SparkSession.builder.getOrCreate()

3.5.3


25/07/01 15:59:24 WARN Utils: Your hostname, TCEU3048M.local resolves to a loopback address: 127.0.0.1; using 192.168.151.110 instead (on interface en0)
25/07/01 15:59:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 15:59:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
!pip install transformers


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, LongType

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Step 1: Create Spark Session
spark = SparkSession.builder.appName("T5Seq2SeqExample").getOrCreate()
sc = spark.sparkContext

# Step 2: Create Example DataFrame
schema = StructType(
    [
        StructField("id", LongType(), nullable=False),
        StructField("sentence", StringType(), nullable=False),
    ]
)

data = [
    Row(1, "It is a good test for Spark."),
    Row(2, "Spark DataFrames are powerful."),
    Row(3, "LLMs could be very slow."),
    Row(4, "It is a naive statement."),
]

input_df = spark.createDataFrame(data, schema=schema)

# Step 3: Load Model & Tokenizer ON DRIVER
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

# Step 4: Broadcast to Workers
broadcast_model = sc.broadcast(model)
broadcast_tokenizer = sc.broadcast(tokenizer)


# Step 5: Define UDF with Decorator
@udf(StringType())
def t5_infer_udf(input_text: str) -> str:
    # Get broadcasted objects
    model = broadcast_model.value
    tokenizer = broadcast_tokenizer.value

    # Build prompt and tokenize
    prompt = f"sentiment of the text: {input_text}"
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate response
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Step 6: Apply UDF
results_df = input_df.withColumn("output_column", t5_infer_udf("sentence"))

# Step 7: Show Results
results_df.show(truncate=False)

# real    0m35.244s
# user    0m4.810s
# sys     0m1.038s

ModuleNotFoundError: No module named 'torch'

In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, LongType
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Create a Spark session
spark = SparkSession.builder.appName("T5Seq2SeqExample").getOrCreate()

# Create an Example Spark DataFrame
schema = StructType(
    [
        StructField("id", LongType(), nullable=False),
        StructField("sentence", StringType(), nullable=False),
    ]
)

data = [
    Row(1, "It is a good test for Spark."),
    Row(2, "Spark DataFrames are powerful."),
    Row(3, "LLMs could be very slow."),
    Row(4, "It is a naive statement."),
]

input_df = spark.createDataFrame(data, schema=schema)

# Loading t5 Model and Tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")


# Defining the Spark UDF
def t5_seq2seq_udf(input_text):
    prompt = f"sentiment of the text: {input_text}"
    input = tokenizer(prompt, return_tensors="pt")
    output = model.generate(**input)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return output_text


t5_udf = udf(t5_seq2seq_udf, returnType=StringType())

results_df = input_df.withColumn("output_column", t5_udf(input_df["sentence"]))

results_df.show(truncate=False)

# real    0m52.339s
# user    0m6.783s
# sys     0m1.317s

# real    1m0.423s
# user    0m7.212s
# sys     0m1.448s

+---+------------------------------+-------------+
|id |sentence                      |output_column|
+---+------------------------------+-------------+
|1  |It is a good test for Spark.  |positive     |
|2  |Spark DataFrames are powerful.|positive     |
|3  |LLMs could be very slow.      |negative     |
|4  |It is a naive statement.      |negative     |
+---+------------------------------+-------------+



In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, LongType

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Step 1: Create Spark Session
spark = SparkSession.builder.appName("T5Seq2SeqExample").getOrCreate()
sc = spark.sparkContext

# Step 2: Create Example DataFrame
schema = StructType(
    [
        StructField("id", LongType(), nullable=False),
        StructField("sentence", StringType(), nullable=False),
    ]
)

data = [
    Row(1, "It is a good test for Spark."),
    Row(2, "Spark DataFrames are powerful."),
    Row(3, "LLMs could be very slow."),
    Row(4, "It is a naive statement."),
]

input_df = spark.createDataFrame(data, schema=schema)

# Step 3: Load Model & Tokenizer ON DRIVER
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

# Step 4: Broadcast to Workers
broadcast_model = sc.broadcast(model)
broadcast_tokenizer = sc.broadcast(tokenizer)


# Step 5: Define UDF with Decorator
def make_t5_udf(model_broadcast, tokenizer_broadcast):
    model = model_broadcast.value
    tokenizer = tokenizer_broadcast.value

    @udf(StringType())
    def inner_udf(input_text: str) -> str:
        prompt = f"sentiment of the text: {input_text}"
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(**inputs)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    return inner_udf


# Create the parameterized UDF
t5_udf = make_t5_udf(broadcast_model, broadcast_tokenizer)

# Apply the UDF
results_df = input_df.withColumn("output_column", t5_udf("sentence"))
results_df.show(truncate=False)

# real    1m19.654s
# user    0m7.219s
# sys     0m1.937s

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
25/06/27 10:47:27 WARN Utils: Your hostname, codespaces-66b7ee resolves to a loopback address: 127.0.0.1; using 10.0.11.83 instead (on interface eth0)
25/06/27 10:47:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/27 10:47:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/27 10:47:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eve

+---+------------------------------+-------------+
|id |sentence                      |output_column|
+---+------------------------------+-------------+
|1  |It is a good test for Spark.  |positive     |
|2  |Spark DataFrames are powerful.|positive     |
|3  |LLMs could be very slow.      |negative     |
|4  |It is a naive statement.      |negative     |
+---+------------------------------+-------------+



In [ ]:
from pyspark.sql import SparkSession, Row
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from pyspark.sql.types import StructType, StructField, LongType, StringType

# Spark session
spark = SparkSession.builder.appName("T5 mapPartitions").getOrCreate()
sc = spark.sparkContext

# Sample data
schema = StructType(
    [
        StructField("id", LongType(), nullable=False),
        StructField("sentence", StringType(), nullable=False),
    ]
)

data = [
    Row(1, "It is a good test for Spark."),
    Row(2, "Spark DataFrames are powerful."),
    Row(3, "LLMs could be very slow."),
    Row(4, "It is a naive statement."),
]

df = spark.createDataFrame(data, schema=schema)

# Load and broadcast model/tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
broadcast_model = sc.broadcast(model)
broadcast_tokenizer = sc.broadcast(tokenizer)


# Function to process a partition
def process_partition(rows):
    model = broadcast_model.value
    tokenizer = broadcast_tokenizer.value

    for row in rows:
        prompt = f"sentiment of the text: {row['sentence']}"
        inputs = tokenizer(prompt, return_tensors="pt")
        output = model.generate(**inputs)
        result = tokenizer.decode(output[0], skip_special_tokens=True)

        yield (row["id"], row["sentence"], result)


# Use mapPartitions
result_rdd = df.rdd.mapPartitions(process_partition)

# Convert to DataFrame
result_df = result_rdd.toDF(["id", "sentence", "output_column"])
result_df.show(truncate=False)

# real    0m38.225s
# user    0m4.914s
# sys     0m1.087s

In [ ]:
# 🔥 Performance Advantage

# Method	Model Load	Processing Granularity	Best For
# @udf	Every row	Per-row	Simplicity
# mapPartitions	Once/partition	Per-partition	Speed, batch inference
# foreachPartition	Once/partition	No return (side-effect)	Saving to sink (DB, S3)

In [ ]:
from pysparky.functions.ai.embedding import t5_infer_udf

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, LongType

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Step 1: Create Spark Session
spark = SparkSession.builder.appName("T5Seq2SeqExample").getOrCreate()
sc = spark.sparkContext

# Example usage of the t5_infer_udf
input_df = spark.createDataFrame(
    [
        Row(1, "It is a good test for Spark."),
        Row(2, "Spark DataFrames are powerful."),
        Row(3, "LLMs could be very slow."),
        Row(4, "It is a naive statement."),
    ],
    schema=StructType(
        [
            StructField("id", LongType(), nullable=False),
            StructField("sentence", StringType(), nullable=False),
        ]
    ),
)

results_df = input_df.withColumn("output_column", t5_infer_udf("sentence"))
results_df.show(truncate=False)

/Users/cenz.wong/pysparky-pyspark-helper/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
25/07/01 16:03:38 WARN Utils: Your hostname, TCEU3048M.local resolves to a loopback address: 127.0.0.1; using 192.168.151.110 instead (on interface en0)
25/07/01 16:03:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 16:03:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+------------------------------+-------------+
|id |sentence                      |output_column|
+---+------------------------------+-------------+
|1  |It is a good test for Spark.  |positive     |
|2  |Spark DataFrames are powerful.|positive     |
|3  |LLMs could be very slow.      |negative     |
|4  |It is a naive statement.      |negative     |
+---+------------------------------+-------------+



ImportError: cannot import name 't5_infer_udf' from 'pysparky.functions.ai.embedding' (/Users/cenz.wong/pysparky-pyspark-helper/pysparky/functions/ai/embedding.py)

25/07/01 17:04:57 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 206473 ms exceeds timeout 120000 ms
25/07/01 17:04:57 WARN SparkContext: Killing executors is not supported by current scheduler.
25/07/01 17:04:58 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
from pysparky.functions.ai.llm import build_text_generation_udf
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, LongType
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Step 1: Create Spark Session
spark = SparkSession.builder.appName("T5Seq2SeqExample").getOrCreate()
sc = spark.sparkContext


# Step 2: Create Example DataFrame
schema = StructType(
    [
        StructField("id", LongType(), nullable=False),
        StructField("sentence", StringType(), nullable=False),
    ]
)

data = [
    Row(1, "It is a good test for Spark."),
    Row(2, "Spark DataFrames are powerful."),
    Row(3, "LLMs could be very slow."),
    Row(4, "It is a naive statement."),
]

input_df = spark.createDataFrame(data, schema=schema)

# Step 3: Load Model & Tokenizer ON DRIVER
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

bc_model = sc.broadcast(model)
bc_tokenizer = sc.broadcast(tokenizer)

# Create the parameterized UDF
t5_sentiment_inference_udf = build_text_generation_udf(
    bc_model, bc_tokenizer, "sentiment of the text"
)

# Apply the UDF
results_df = input_df.withColumn(
    "output_column", t5_sentiment_inference_udf("sentence")
)
results_df.show(truncate=False)

bc_model.destroy()
bc_tokenizer.destroy()

+---+------------------------------+-------------+
|id |sentence                      |output_column|
+---+------------------------------+-------------+
|1  |It is a good test for Spark.  |positive     |
|2  |Spark DataFrames are powerful.|positive     |
|3  |LLMs could be very slow.      |negative     |
|4  |It is a naive statement.      |negative     |
+---+------------------------------+-------------+

